In [5]:
import os
import cv2
import numpy as np
from skimage.morphology import skeletonize

def find_endpoint(skeleton):
    connectivity = find_connectivity(skeleton)
    # print(np.unique(connectivity, return_counts=True))
    return np.argwhere(connectivity == 2)

def find_connectivity(binary_image):
    result = np.zeros_like(binary_image, dtype=np.uint8)
    for [i, j] in np.argwhere(binary_image):
        conn_count = 0
        for ii in range(i-1, i+2):
            for jj in range(j-1, j+2):
                if binary_image[ii, jj]:
                    conn_count += 1
        result[i, j] = conn_count
    return result

def find_tips(image, boundary=0.1):
    skeleton = skeletonize(image)
    endpoints = find_endpoint(skeleton)
    tips = []
    for [i, j] in endpoints:
        if i < boundary*image.shape[0] or i > (1-boundary)*image.shape[0] or j < boundary*image.shape[1] or j > (1-boundary)*image.shape[1]:
            continue
        tips.append((i, j))
    return tips

In [11]:
from tqdm import tqdm

def composite(image: np.ndarray, mask: np.ndarray, endpoints: tuple, alpha=0.7, speed=None):
    '''
    Note that endpoint coordinates are in (y, x) format
    '''
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask_bgr = np.where(mask_bgr == (255, 255, 255), (0, 255, 0), mask_bgr).astype(np.uint8)
    overlay = cv2.addWeighted(image, 1, mask_bgr, alpha, 0)
    title = f"Estimated tip position: {endpoints if len(endpoints) < 2 else 'Multi'}, Speed: " + ("None" if speed is None else f"{speed:.2f}") + " Pixel/s"
    cv2.putText(overlay, title , (40,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 205), 2)
    for endpoint in endpoints:
        cv2.circle(overlay, (endpoint[1], endpoint[0]), 5, (128, 0, 245), -1)
    return overlay


In [17]:
segmentation_folder = "Data/Datasets/RigidModelVideo-11-21/11-21-1-clip2/outputs_f_t_100/segmentation/filled_5_pruned"
image_folder = "Data/Datasets/RigidModelVideo-11-21/11-21-1-clip2"
output_folder = f"{segmentation_folder}/composite"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

fps = 30

last_tip = None
last_index = -1
for i, file in tqdm(enumerate(sorted(os.listdir(segmentation_folder)))):
    if file.endswith(".jpg"):
        segmentation = (cv2.imread(os.path.join(segmentation_folder, file), cv2.IMREAD_GRAYSCALE) > 10).astype(np.uint8) * 255 #deal with jpg compression artifacts
        image = cv2.imread(os.path.join(image_folder, file))  
        tips = find_tips(segmentation)
        speed = None
        if len(tips) == 1:
            tip = tips[0]
            if last_tip is not None:
                speed = np.linalg.norm(np.array(tip) - np.array(last_tip)) * fps / (i - last_index)
            last_tip = tip
            last_index = i
        cv2.imwrite(os.path.join(output_folder, file), composite(image, segmentation, tips, speed=speed))
            

316it [00:29, 10.76it/s]


In [16]:

os.system(f"ffmpeg -framerate 12 -start_number -i {os.getcwd()}/{output_folder}/frame%04d.jpg -r 15 -b:v 20M {os.getcwd()}/{output_folder}/composite.mp4")

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda/conda-bld/ffmpeg_1597178665428/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeh --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libsw

256